The aim is toread 30 year daily rainfall data
1) plot PDF of rainfall for each longitude : at least for average of 10degrees ... so 36 PDFs for the globe (do this also of daily temperature)
2) compute ITCZ movement : location of centroid and its displacement sectorally
3) compute width of ITCZ
=> expectation: more signal over Land


It is copied from WK spectra code. ... just to see how data is loaded and read. Reset can be deleted


Atwood et al 2020: ITCZ centroid - the latitude at which the mean annual area-weighted tropical rainfall to the north equals that to the south, within the bounds 20°N to 20°S

I think this is same as what aiko voight's github code gives: check

In [ ]:
import numpy as np
import xarray as xr
# our local module:
import itcz

import matplotlib as mpl
import matplotlib.pyplot as plt

In [ ]:
import xarray as xr
from pathlib import Path
import myfunctions as mf

In [ ]:
# =========================
# User-defined metadata
# =========================

varname = "pr"

# =========================
# Base CEDA paths
# =========================

CEDA_BASE = Path("/badc/cmip6/data/CMIP6")

In [ ]:
#Model Names
MODELS = {
    "UKESM1-0-LL":  {"institution": "MOHC",         "ensemble": "r1i1p1f2",  "grid": "gn",},
    "CNRM-ESM2-1":  {"institution": "CNRM-CERFACS", "ensemble": "r1i1p1f2",  "grid": "gr",},
    "MPI-ESM1-2-LR":{"institution": "MPI-M",        "ensemble": "r1i1p1f1",  "grid": "gn",},
    "CESM2-WACCM":  {"institution": "NCAR",         "ensemble": "r1i1p1f1",  "grid": "gn",},
    "IPSL-CM6A-LR": {"institution": "IPSL",         "ensemble": "r1i1p1f1",  "grid": "gr",},
}


In [ ]:
#Experiment details
EXPERIMENTS = {
    "HIST":     {"project": "CMIP",        "scenario": "historical", "color": "black"},
    "SSP245":   {"project": "ScenarioMIP", "scenario": "ssp245"},
    "SSP585":   {"project": "ScenarioMIP", "scenario": "ssp585"},
    "G6solar":  {"project": "GeoMIP",      "scenario": "G6solar"},
    "G6sulfur": {"project": "GeoMIP",      "scenario": "G6sulfur"},
}

In [ ]:
#Compute ITCZ shift

itcz_by_exp = {}

for model_name, model_meta in MODELS.items():
    var = {}
    var_by_year = {}

    # --- LOAD DATA FOR A MODEL ---
    for exp, meta in EXPERIMENTS.items():
        if exp not in ["SSP245", "G6sulfur", "G6solar"]:
            continue

        # open dataset
        # --- special-case ensemble override ---
        if model_name == "CESM2-WACCM":
            if meta["scenario"] == "G6sulfur":
                ensemble = "r1i1p1f2"
            else:
                ensemble = "r1i1p1f1"
        else:
            ensemble = model_meta["ensemble"]


        base = (
            CEDA_BASE
            / meta["project"]
            / model_meta["institution"]
            / model_name
            / meta["scenario"]
            / ensemble
            / "Amon"
            / varname
            / model_meta["grid"]
            / "latest"
        )

        print(str(base))
        # ds = mf.open_files(str(base))
        if model_name == "CESM2-WACCM":
            if meta["scenario"] == "G6sulfur":
                ds = mf.open_files_CESM_G6sulfur(base)
            elif meta["scenario"] == "ssp585":
                ds = mf.open_files_CESM_ssp585(base)
            else:
                ds = mf.open_files(str(base))
        elif model_name == "IPSL-CM6A-LR":
            if meta["scenario"] == "ssp585":
                ds = mf.open_files_IPSL_ssp585(base)
            else:
                ds = mf.open_files(str(base))
        else:
            ds = mf.open_files(str(base))



        
        # ds = ...
        var[exp] = mf.read_var(ds, varname)

    # --- SEASONAL MEAN FOR THIS MODEL --- 1-12 for ANN, 6-9 for JJAS, 12,2 for DJF
    for exp, da in var.items():
        var_by_year[exp] = {
            "SEASON": mf.seasonal_mean_by_year_old(da, 1, 12)
        }
        londim = mf.get_lon_dim(da)

    # --- compute ITCZ for this model & store ---
    itcz_by_exp[model_name] = {}
    
    for exp in ["SSP245", "G6sulfur", "G6solar"]:
    
        da_exp = var_by_year[exp]["SEASON"].sel(year=slice(2021, 2050))
        # da_exp dims: (year, lat, lon)
    
        itcz_vals = []
    
        for yr in da_exp.year.values:
            itcz_lon = []
    
            for lo in da_exp.lon.values:
                pr_lat = da_exp.sel(year=yr, lon=lo)
    
                itcz_lat = itcz.get_itczposition_voigt(
                    pr_lat.values,   # 1D (lat)
                    pr_lat.lat.values,
                    latboundary=20,
                    dlat=0.1
                )
    
                itcz_lon.append(itcz_lat)
    
            itcz_vals.append(itcz_lon)
    
        itcz_by_exp[model_name][exp] = xr.DataArray(
            itcz_vals,
            coords={
                "year": da_exp.year,
                "lon": da_exp.lon
            },
            dims=("year", "lon"),
            name="itcz_lat"
        )


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Models
MODELS = {
    "UKESM1-0-LL":  {"institution": "MOHC",         "ensemble": "r1i1p1f2",  "grid": "gn",},
    "CNRM-ESM2-1":  {"institution": "CNRM-CERFACS", "ensemble": "r1i1p1f2",  "grid": "gr",},
    "MPI-ESM1-2-LR":{"institution": "MPI-M",        "ensemble": "r1i1p1f1",  "grid": "gn",},
    "CESM2-WACCM":  {"institution": "NCAR",         "ensemble": "r1i1p1f1",  "grid": "gn",},
    "IPSL-CM6A-LR": {"institution": "IPSL",         "ensemble": "r1i1p1f1",  "grid": "gr",},
}

# Experiments
exps = ["G6sulfur", "G6solar"]

# Prepare figure: rows = models, cols = experiments
fig, axes = plt.subplots(
    nrows=len(MODELS), ncols=2,
    figsize=(14, 1.5*len(MODELS)),
    subplot_kw=dict(projection=ccrs.PlateCarree())
)

# Ensure axes is 2D array for easy indexing
axes = np.atleast_2d(axes)

lat = da["lat"].values
lon = da["lon"].values

# Wrap longitudes for Cartopy (-180 to 180)
lon_wrap = ((lon + 180) % 360) - 180
sort_idx = np.argsort(lon_wrap)
lon_wrap_sorted = lon_wrap[sort_idx]

# Dummy background (for projection)
dummy = np.zeros((lat.size, lon.size))

for i, (model_name, meta) in enumerate(MODELS.items()):
    for j, exp in enumerate(exps):
        ax = axes[i, j]

        # Compute ITCZ shift: current model
        shift = itcz_by_exp[model_name][exp] - itcz_by_exp[model_name]["SSP245"]

        # mean & std across years
        mean_shift = shift.mean(dim="year").values
        std_shift  = shift.std(dim="year").values

        # Wrap & sort ITCZ longitudes
        itcz_lon_wrap = ((itcz_by_exp[model_name][exp][londim].values + 180) % 360) - 180
        sort_idx_itcz = np.argsort(itcz_lon_wrap)
        itcz_mean_sorted = mean_shift[sort_idx_itcz]
        itcz_std_sorted  = std_shift[sort_idx_itcz]
        itcz_lon_sorted  = np.sort(itcz_lon_wrap)

        # Dummy background for map
        lon2d, lat2d = np.meshgrid(lon_wrap_sorted, lat)
        ax.pcolormesh(lon2d, lat2d, dummy, alpha=0, shading='auto', transform=ccrs.PlateCarree())

        # ITCZ shift bars
        ax.bar(
            itcz_lon_sorted,
            itcz_mean_sorted*5,  # optional scaling
            yerr=itcz_std_sorted,
            capsize=2,
            alpha=0.8,
            width=(lon_wrap_sorted[1]-lon_wrap_sorted[0]),
            color="C1",
        )

        # Equator line
        ax.axhline(0, color="k", linewidth=0.5, linestyle="--")

        # Land/ocean
        ax.add_feature(cfeature.LAND, facecolor='lightgray')
        ax.add_feature(cfeature.OCEAN, facecolor='aliceblue')
        ax.coastlines(linewidth=0.6)

        # Global extent
        ax.set_xlim(-180, 180)
        ax.set_ylim(-30, 30)
        ax.set_title(f"{model_name} - {exp}")
        ax.set_ylabel("Lat shift (deg)")

# X-axis label only on bottom row
for ax in axes[-1, :]:
    ax.set_xlabel("Longitude (deg)")

plt.tight_layout()

#Saving figure
fig_directory='/home/users/bidyut/20260112_Basic_Analysis'
figname="Figure_ITCZ_Adam_shiftx5_AnnualMean2021-2050"

# plt.savefig(''+str(fig_directory)+'/'+str(figname)+'.svg', format="svg",transparent=True, dpi=1200)
# !rsvg-convert -f pdf -o {fig_directory}/{figname}.pdf {fig_directory}/{figname}.svg

plt.savefig(''+str(fig_directory)+'/'+str(figname)+'.pdf', format="pdf")

plt.show()


In [ ]:
import numpy as np
import xarray as xr

# experiments relative to SSP245
experiments = ["G6sulfur", "G6solar"]

ITCZ_shift_all_2021_2050 = {}

for model in MODELS:
    ITCZ_shift_all_2021_2050[model] = {}

    for exp in experiments:
        # --- ITCZ shift relative to SSP245 ---
        shift = itcz_by_exp[model][exp] - itcz_by_exp[model]["SSP245"]
        # dims typically: (year, lon)

        # --- mean over years ---
        shift_mean = shift.mean(dim="year")

        # --- longitude coordinate ---
        lon = shift[londim]

        # --- global mean ITCZ shift (scalar) ---
        mean_shift_scalar = shift_mean.mean(dim=londim)

        ITCZ_shift_all_2021_2050[model][f"{exp}-SSP245"] = {
            "mean": mean_shift_scalar,
            "by_lon": shift_mean
        }

In [ ]:
%store ITCZ_shift_all_2021_2050

In [ ]:
import matplotlib.pyplot as plt

# Pick a model and experiment
model = "UKESM1-0-LL"
exp = "G6sulfur-SSP245"

# Extract data
mean_shift = ITCZ_shift_all_2021_2050[model][exp]["mean"].values  # scalar
by_lon    = ITCZ_shift_all_2021_2050[model][exp]["by_lon"]        # DataArray (lon)
lon       = by_lon[londim].values

# Plot
plt.figure(figsize=(10,4))
plt.plot(lon, by_lon, 'o', alpha=0.6, label='Longitude-resolved')
plt.hlines(mean_shift, lon.min(), lon.max(), colors='r', linewidth=2, label='Mean ITCZ shift')
plt.xlabel("Longitude (deg)")
plt.ylabel("ITCZ shift (deg)")
plt.title(f"{model} - {exp} ITCZ shift")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import numpy as np

# Pick a model and experiment
model = "UKESM1-0-LL"
exp = "G6sulfur-SSP245"

# Extract data
mean_shift = ITCZ_shift_all_2021_2050[model][exp]["mean"].values  # scalar
by_lon    = ITCZ_shift_all_2021_2050[model][exp]["by_lon"]        # DataArray (lon)
lon       = by_lon[londim].values
lat_dummy = np.array([0])  # dummy latitude (1D)

# Wrap longitudes for Cartopy (-180 to 180)
lon_wrap = ((lon + 180) % 360) - 180
sort_idx = np.argsort(lon_wrap)
lon_wrap_sorted = lon_wrap[sort_idx]
by_lon_sorted = by_lon[sort_idx]

# Create figure with PlateCarree
fig, ax = plt.subplots(figsize=(12,4), subplot_kw=dict(projection=ccrs.PlateCarree()))

# Make 2D grids for pcolormesh
lon2d, lat2d = np.meshgrid(lon_wrap_sorted, lat_dummy)
z2d = np.zeros_like(lon2d)

# Dummy background (required for Cartopy)
ax.pcolormesh(lon2d, lat2d, z2d, alpha=0, shading='auto', transform=ccrs.PlateCarree())

# Plot ITCZ shift as bars
ax.bar(
    lon_wrap_sorted,
    by_lon_sorted*5,  # optional scaling
    alpha=0.7,
    width=(lon_wrap_sorted[1]-lon_wrap_sorted[0]),
    color="C1",
    label="Longitude-resolved"
)

# Plot mean shift line
ax.axhline(mean_shift*5, color='r', linewidth=2, label='Mean shift')

# Land/ocean and equator
ax.add_feature(cfeature.LAND, facecolor='lightgray')
ax.add_feature(cfeature.OCEAN, facecolor='aliceblue')
ax.coastlines()
ax.axhline(0, color="k", linestyle="--", linewidth=0.5)

# Map extent
ax.set_xlim(-180, 180)
ax.set_ylim(-10, 10)

ax.set_title(f"{model} - {exp} ITCZ shift")
ax.set_xlabel("Longitude (deg)")
ax.set_ylabel("Latitude shift (deg * 5)")  # scaled as above
# ax.legend()
plt.show()